In [0]:
from pyspark.sql.functions import (col, trim, initcap, lower, regexp_replace, when)

In [0]:
%sql
create schema if not exists ecommerce.silver

In [0]:
# Lendo DataFrame da tabela clientes na camada silver

clientes_df = spark.table("ecommerce.bronze.clientes")

In [0]:
clientes_df = (
    clientes_df
    # Selecionar colunas
    .select(
        col("cliente_id"),
        col("nome"),
        col("email"),
        col("cpf"),
        col("data_nascimento"),
        col("cidade"),
        col("estado"),
        col("cep"),
        col("ativo").alias("cadastro"),
        col("data_cadastro"),
        col("telefone")
    )

    # Nome em Title Case
    .withColumn("nome", initcap(trim(col("nome"))))

    # Email em lowercase
    .withColumn("email", lower(trim(col("email"))))

    # Limpeza do telefone (remove tudo que não é número)
    .withColumn(
        "telefone_num",
        regexp_replace(col("telefone"), "[^0-9]", "")
    )

    # Formatação do telefone: (XX) XXXXX-XXXX
    .withColumn(
        "telefone",
        when(
            col("telefone_num").rlike("^[0-9]{11}$"),
            regexp_replace(
                col("telefone_num"),
                "([0-9]{2})([0-9]{5})([0-9]{4})",
                "(\\1) \\2-\\3"
            )
        )
    )

    # Criação da região baseada no estado
    .withColumn(
        "regiao",
        when(col("estado").isin("SP", "RJ", "MG", "ES"), "Sudeste")
        .when(col("estado").isin("PR", "SC", "RS"), "Sul")
        .when(col("estado").isin("BA", "PE", "CE", "RN", "PB", "AL", "SE", "MA", "PI"), "Nordeste")
        .when(col("estado").isin("AM", "PA", "AC", "RO", "RR", "AP", "TO"), "Norte")
        .when(col("estado").isin("GO", "MT", "MS", "DF"), "Centro-Oeste")
        .otherwise("Indefinido")
    )
)


In [0]:
#Salvando DataFrame na camada silver
clientes_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("ecommerce.silver.clientes")